# Парсинг информации о странах и их рейтинга

## Подключение нужных библиотек

In [2]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from tqdm import tqdm

##  Парсинг стран и их рейтинга в Happiness Score

In [10]:
def get_happiness_data(url = "https://en.wikipedia.org/wiki/World_Happiness_Report"):
    """
    Собирает информацию о странах и их рейтинге в Happiness Score с Википедии
    Возвращает: DataFrame
    """

    try:
        happiness_df = pd.read_html(url)[15]
    except Exception as e:
        print(f"Error reading HTML tables: {e}")
        return pd.DataFrame()

    happiness_df.rename(columns={'Country or region': 'Country',
                                 'Overall rank': 'Happiness Rank',
                                 'Life evaluation': 'Happiness Score'},
                        inplace=True)
    happiness_df = happiness_df[['Country', 'Happiness Rank', 'Happiness Score']]
    return happiness_df

In [15]:
countries_happiness_df = get_happiness_data()

os.makedirs('../data', exist_ok=True)
countries_happiness_df.to_csv('../data/countries_happiness.csv', index=False)

## Парсинг различных показателей стран с World Bank Open Data

In [16]:
BASE_URL = "https://data.worldbank.org"

def get_country_links(base_url=BASE_URL):
    """
    Достает ссылки на все страны из подкаталога country
    Возвращает массив пар: (Название страны, Ссылка на страну)
    """
    url = urljoin(base_url, "country")
    print(f"Ищем ссылки на страны на странице: {url}")
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")
    country_links = []

    # Find all <a> tags whose href begins with "/country/"
    for a in soup.find_all("a", href=True):
        href = a['href']
        if href.startswith("/country/"):
            country_name = a.get_text(strip=True)
            full_url = urljoin(base_url, href)
            # Avoid duplicates
            if country_name and full_url not in [link for (_, link) in country_links]:
                country_links.append((country_name, full_url))

    print(f"Найдено {len(country_links)} стран.")
    return country_links

In [17]:
def get_country_indicators(country_url):
    """
    Принимает URL страницы страны (например, "https://data.worldbank.org/country/finland?view=chart")
    и возвращает:
      - код страны
      - словарь с данными индикаторов: {название индикатора: значение (float или None)},
      - словарь с метаданными: {название индикатора: ссылка на индикатор}.
    """

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(country_url)

    wait = WebDriverWait(driver, 5)
    try:
        # Ищем вкладку "By Theme" по XPath (элемент <li> с классом react-tabs__tab, внутри которого есть <span> с
        # нужным текстом)
        by_theme_tab = wait.until(
            ec.element_to_be_clickable(
                (By.XPATH, "//li[contains(@class, 'react-tabs__tab') and .//span[contains(text(), 'By Theme')]]")
            )
        )
        # Кликаем на нужную вкладку
        driver.execute_script("arguments[0].click();", by_theme_tab)

        # Ждем появления контента вкладки
        wait.until(ec.presence_of_element_located((By.CSS_SELECTOR, ".indicator-item")))
        time.sleep(2)  # небольшая задержка
        html = driver.page_source
        # print(f"Вкладка 'By Theme' успешно открыта для {country_url}")
    except Exception as e:
        # print(f"Не удалось найти или кликнуть по вкладке 'By Theme' для {country_url}: {e}")
        driver.quit()
        return None, {}, {}
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    indicators_info = soup.find_all(class_='indicator-item__inner')

    indicator_values = {}
    indicator_links = {}
    country_code = None

    for indicator_info in indicators_info:
        headline_elem = indicator_info.find(class_='indicator-item__headline-mobile')
        if headline_elem:
            a_tag = headline_elem.find('a')
            if a_tag:
                indicator_name = a_tag.get_text(strip=True)
                indicator_link = a_tag.get('href')
                indicator_link, country_code = indicator_link.split('?locations=')
                indicator_links[indicator_name] = indicator_link

                data_info = indicator_info.find(class_='indicator-item__data-info')
                if data_info:
                    span_tag = data_info.find('span')
                    if span_tag:
                        try:
                            # Убираем запятые, пробуем преобразовать текст в число
                            value = float(span_tag.get_text(strip=True).replace(',', ''))
                        except Exception as ex:
                            print(f"Ошибка преобразования значения для '{indicator_name}': {ex}")
                            value = None
                    else:
                        value = None
                else:
                    value = None
                indicator_values[indicator_name] = value

    return country_code, indicator_values, indicator_links

### Собираем информацию по всем индикаторам для всех стран

In [18]:
countries = get_country_links()

all_country_data = []  # список словарей: каждый словарь содержит данные для одной страны
metadata_dict = {}     # метаданные (предполагается, что структура индикаторов одинакова для всех стран)
collected_countries = set() # названия стран, информация про которые уже собрана, чтобы пропускать их при повторном запуске

Ищем ссылки на страны на странице: https://data.worldbank.org/country
Найдено 217 стран.


In [20]:
for country_name, country_url in (pbar := tqdm(countries)):
    pbar.set_description(f"Собираем информацию о стране {country_name}")

    if country_name in collected_countries:
        continue

    attempt = 0
    wait_time = 10
    # Первая попытка получения данных
    country_code, indicators, metadata = get_country_indicators(country_url)
    # Если список индикаторов пуст, делаем повторные попытки до 10 раз
    while len(indicators) == 0 and attempt < 10:
        attempt += 1
        pbar.write(f"Попытка {attempt} для {country_name} неуспешна. Ожидание {wait_time} секунд перед повторной попыткой...")
        time.sleep(wait_time)
        wait_time += 10
        country_code, indicators, metadata = get_country_indicators(country_url)

    if len(indicators) == 0:
        pbar.write(f"Не удалось получить данные для {country_name} после 10 попыток.")
    else:
        collected_countries.add(country_name)
    # Приводим ссылки индикаторов к полному виду
    metadata = {indicator_name: urljoin(BASE_URL, metadata[indicator_name])
                for indicator_name in metadata.keys()}

    # Формируем словарь с данными: имя страны + индикаторы
    data_entry = {"Country": country_name, "Country Code": country_code}
    data_entry.update(indicators)
    all_country_data.append(data_entry)

    # Обновляем общий словарь метаданных
    metadata_dict.update(metadata)

    time.sleep(2)  # пауза между запросами к разным странам

Собираем информацию о стране Brunei Darussalam:  13%|█▎        | 28/217 [05:16<1:01:02, 19.38s/it]     

Попытка 1 для Brunei Darussalam неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Bulgaria:  13%|█▎        | 29/217 [06:12<1:33:43, 29.91s/it]         

Попытка 1 для Bulgaria неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Costa Rica:  21%|██        | 46/217 [12:25<47:56, 16.82s/it]                

...

Собираем информацию о стране Tonga:  90%|████████▉ | 195/217 [1:13:38<06:57, 18.96s/it]               

Попытка 1 для Tonga неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Uganda:  93%|█████████▎| 202/217 [1:16:39<04:48, 19.24s/it]                  

Попытка 1 для Uganda неуспешна. Ожидание 10 секунд перед повторной попыткой...


Собираем информацию о стране Zimbabwe: 100%|██████████| 217/217 [1:22:32<00:00, 22.82s/it]             


### Объединяем всю информацию в один DataFrame и сохраняем

In [21]:
os.makedirs('../data', exist_ok=True)

# Создаем DataFrame, где индекс – название страны, а столбцы – названия индикаторов
df = pd.DataFrame(all_country_data)
df.set_index("Country", inplace=True)
df.to_csv("../data/countries_indicators.csv", encoding="utf-8")
print("Сохранены данные по индикаторам для стран в файле '../data/country_indicators.csv'.")

# Сохраняем метаданные в отдельный CSV: для каждого индикатора его ссылка
meta_df = pd.DataFrame(list(metadata_dict.items()), columns=["Indicator", "Link"])
meta_df.to_csv("../data/indicators_metadata.csv", index=False, encoding="utf-8")
print("Сохранены метаданные индикаторов в файле '../data/indicators_metadata.csv'.")

Сохранены данные по индикаторам для стран в файле '../data/country_indicators.csv'.
Сохранены метаданные индикаторов в файле '../data/indicators_metadata.csv'.


## Собираем данные о странах с помощью REST Countries API

### Получаем всю информацию о всех странах

In [57]:
url = "https://restcountries.com/v3.1/all"
response = requests.get(url)

if response.status_code == 200:
    countries = response.json()
else:
    print("Error:", response.status_code)
    exit()

### Выбираем нужную информацию и сохраняем

In [60]:
country_list = []
for country in countries:
    name = country.get("name", {}).get("common", None)
    country_code = country.get("cca2", None)
    country_code_cca3 = country.get("cca3", None)
    region = country.get("region", None)
    languages = list(country.get("languages", {}).values()) if country.get("languages") else None
    main_language = languages[0] if languages else None
    independent = country.get("independent", None)
    un_member = country.get("unMember", None)
    currencies = country.get("currencies", {})
    main_currency = list(currencies.keys())[0] if currencies else None
    population = country.get("population", None)
    area = country.get("area", None)
    car_side = country.get("car", {}).get("side", None)

    country_list.append({
        "Country": name,
        "Country Code": country_code,
        "Country Code CCA3": country_code_cca3,
        "Region": region,
        "Main Language": main_language,
        "Independent": independent,
        "UN Member": un_member,
        "Main Currency": main_currency,
        "Population": population,
        "Area": area,
        "Car Side": car_side,
    })

# Сохраняем в CSV
df = pd.DataFrame(country_list)
os.makedirs('../data', exist_ok=True)
df.to_csv("../data/countries_base_info.csv", index=False)

print("Data saved to '../data/countries_base_info.csv'")

Data saved to '../data/countries_base_info.csv'


## Объединяем все данные

In [4]:
happiness_df = pd.read_csv('../data/countries_happiness.csv')
indicators_df = pd.read_csv('../data/countries_indicators.csv')
base_info_df = pd.read_csv('../data/countries_base_info.csv')

Убираем колонки без названий

In [14]:
unnamed_cols = indicators_df.columns[
    indicators_df.columns.str.contains('unnamed', case = False)]
indicators_df.drop(columns=unnamed_cols, inplace=True)

In [15]:
# base_info и indicators можно объединить по коду страны, поскольку коды совпадают даже если не совпадают названия стран
merged_df = pd.merge(base_info_df, indicators_df.drop(columns='Country'), on='Country Code', how='inner')

Посмотрим на страны которые есть в рейтинге счастья, но нет в собранном датасете

In [16]:
print('\n'.join(set(happiness_df['Country']) - set(merged_df['Country'])))

Taiwan
Congo (Kinshasa)
State of Palestine
Congo (Brazzaville)


* Congo - есть два разных Congo и они по-разному записаны в этих датасетах - переименуем
* State of Palestine - записано как просто Palestine - переименуем
* Taiwan - не признается Global Bank, считается частью Китая - удалим

In [17]:
happiness_df['Country'] = happiness_df['Country'].replace({
    'State of Palestine': 'Palestine',
    'Congo (Kinshasa)': 'DR Congo',
    'Congo (Brazzaville)': 'Republic of the Congo'
})

In [18]:
final_df = pd.merge(happiness_df, merged_df, on='Country', how='right')

### Добавим еще политический режим по данным  [Our World in Data](https://ourworldindata.org/grapher/political-regime?time=latest#explore-the-data)

In [19]:
regime_df = pd.read_csv('../data/countries_political_regime.csv').rename(
    {'Code': 'Country Code CCA3', 'Political regime': 'Political Regime'},
    axis=1)

# Перекодируем номер политического режима в название
regime_df['Political Regime'] = regime_df['Political Regime'].replace({
    0: 'Closed Autocracies',
    1: 'Electoral Autocracies',
    2: 'Electoral Democracies',
    3: 'Liberal Democracies'
})

regime_df = regime_df[['Country Code CCA3', 'Political Regime']]

In [20]:
final_df = pd.merge(final_df, regime_df, on='Country Code CCA3', how='left')

### Финальная обработка и сохранение итога в таблицу EXCEL и CSV файл

Уберем ненужное

In [21]:
final_df.drop(columns=['Happiness Rank'], inplace=True)
final_df.sort_values(by='Happiness Score', ascending=False, inplace=True)

Немного изменим порядок столбцов

In [22]:
start_index = final_df.columns.get_loc('Independent')
first_part = final_df.columns[:start_index]
middle_part = final_df.columns[start_index:-1]
last_column = final_df.columns[-1]

# Новый порядок столбцов
new_order = list(first_part) + [last_column] + list(middle_part)

# Перестановка столбцов
final_df = final_df[new_order]

Наконец сохраняем

In [23]:
final_df.to_csv('../data/countries_full_data.csv', index=False)
final_df.to_excel('../data/countries_full_data.xlsx', index=False)


<ipython-input-23-477ac2503e72>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  final_df.to_excel('../data/countries_full_data.xlsx', index=False)
